# MLP

In [28]:
import numpy as np
import time
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import tensorflow as tf

In [29]:
class PerceptronMulticapa:
    def __init__(self, capas, activaciones, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.activaciones = activaciones
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa (He para ReLU, Xavier para otras)
            if activaciones[i] == "relu":
                peso = np.random.randn(capas[i], capas[i+1]) * np.sqrt(2.0 / capas[i])
            else:
                peso = np.random.randn(capas[i], capas[i+1]) * np.sqrt(1.0 / capas[i])
            self.pesos.append(peso)
            bias = np.zeros(capas[i+1])
            self.bias.append(bias)

    def activacion(self, x, activacion):
        match activacion:
            case "tanh":
                return np.tanh(x)
            case "sigmoid":
                return 1 / (1 + np.exp(-x))
            case "relu":
                return np.maximum(0, x)
            case "lineal":
                return x
            case "softmax":
                # suponemos entrada 1D por muestra
                e_x = np.exp(x - np.max(x))
                return e_x / np.sum(e_x)

    def activacion_derivada(self, x, activacion):
        # Aquí 'x' es la activación (no el pre-activation z) — coincide con tu uso actual.
        match activacion:
            case "tanh":
                return 1 - x ** 2
            case "sigmoid":
                return x * (1 - x)
            case "relu":
                return (x > 0).astype(float)
            case "lineal":
                return np.ones_like(x)
            case "softmax":
                # Si usas softmax + cross-entropy, la derivada se maneja en el delta (y_pred - y),
                # así que no la usamos aquí; devolvemos 1s para no romper formas si se usa.
                return np.ones_like(x)

    def feedforward(self, X):
        # Calcular la salida de cada capa (lista de activaciones por capa)
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x, self.activaciones[i])
            capa_activacion.append(y)
        return capa_activacion

    def backpropagation(self, X, y, capa_activacion):
        # delta de la capa de salida
        if self.activaciones[-1] == "softmax":
            delta_out = capa_activacion[-1] - y
        else:
            error = capa_activacion[-1] - y
            delta_out = error * self.activacion_derivada(capa_activacion[-1], self.activaciones[-1])

        # Lista de deltas alineada con self.pesos: deltas[i] corresponde al delta de a[i+1]
        n_pesos = len(self.pesos)
        deltas = [None] * n_pesos
        deltas[-1] = delta_out

        # Propagar hacia atrás los deltas para capas ocultas
        # para i = n_pesos-2 .. 0
        for i in range(n_pesos - 2, -1, -1):
            # deltas[i+1] tiene forma (n_{i+2},) => np.dot(..., pesos[i+1].T) -> (n_{i+1},)
            deltas[i] = np.dot(deltas[i+1], self.pesos[i+1].T) * self.activacion_derivada(
                capa_activacion[i+1], self.activaciones[i]
            )

        # Actualizar pesos y bias
        for i in range(n_pesos):
            a = capa_activacion[i].reshape(-1, 1)        # (n_i,1)
            d = deltas[i].reshape(1, -1)                # (1, n_{i+1})
            self.pesos[i] -= self.alpha * np.dot(a, d)  # (n_i, n_{i+1})
            self.bias[i] -= self.alpha * deltas[i]

    def entrenar(self, X, y, epochs):
        inicio = time.time()
        for epoch in range(epochs):
            # Entrenamiento por cada muestra
            for i in range(len(X)):
                capa_activacion = self.feedforward(X[i])
                self.backpropagation(X[i], y[i], capa_activacion)

            # Cada 10 épocas, imprimir progreso
            if (epoch + 1) % 10 == 0:
                # Calcular loss y accuracy sobre el conjunto de entrenamiento
                losses = []
                aciertos = 0
                for i in range(len(X)):
                    pred = self.predecir(X[i])
                    if self.activaciones[-1] == "softmax":
                        # Cross-entropy loss
                        # Agregamos pequeña constante para estabilidad numérica
                        loss = -np.sum(y[i] * np.log(pred + 1e-9))
                    else:
                        # MSE loss
                        loss = np.mean((pred - y[i]) ** 2)
                    losses.append(loss)
                    if np.argmax(pred) == np.argmax(y[i]):
                        aciertos += 1
                loss_prom = np.mean(losses)
                acc = aciertos / len(X)

                print(f"Epoch {epoch+1}/{epochs} completado | Loss: {loss_prom:.4f} | Accuracy: {acc*100:.2f}%")
        
        fin = time.time()
        return fin - inicio

    def predecir(self, X):
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]

## Cargar MNIST

In [30]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype(np.float32)
y = mnist.target.astype(int)

# Normalizar a rango [0,1]
X /= 255.0


## Feature Engineering

In [31]:
# Estandarizar los datos (media 0, varianza 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encoding de las etiquetas
Y = np.eye(10)[y]

# Dividir en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=y
)


## Crear y entrenar la red

In [32]:
perceptron = PerceptronMulticapa(capas=[784, 128, 64, 10], activaciones=["sigmoid","sigmoid", "softmax"], alpha=0.01)
tiempo_entrenamiento = perceptron.entrenar(X_entrenamiento, y_entrenamiento, epochs=50)

# Evaluar con un subconjunto (por tiempo)
predicciones = []
y_reales = []

for i in range(500):
    salida = perceptron.predecir(X_prueba[i])
    pred_clase = np.argmax(salida)
    real_clase = np.argmax(y_prueba[i])
    predicciones.append(pred_clase)
    y_reales.append(real_clase)

Epoch 10/50 completado | Loss: 0.0095 | Accuracy: 99.86%
Epoch 20/50 completado | Loss: 0.0016 | Accuracy: 100.00%
Epoch 30/50 completado | Loss: 0.0008 | Accuracy: 100.00%
Epoch 40/50 completado | Loss: 0.0005 | Accuracy: 100.00%
Epoch 50/50 completado | Loss: 0.0004 | Accuracy: 100.00%


## Metricas

In [33]:
accuracy = accuracy_score(y_reales, predicciones)
precision = precision_score(y_reales, predicciones, average='macro')
recall = recall_score(y_reales, predicciones, average='macro')
f1 = f1_score(y_reales, predicciones, average='macro')

print("\nResultados del modelo:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"Tiempo de entrenamiento: {tiempo_entrenamiento:.2f} segundos")


Resultados del modelo:
Accuracy:  0.9640
Precision: 0.9625
Recall:    0.9643
F1-score:  0.9630
Tiempo de entrenamiento: 646.76 segundos


## Parte 2

In [34]:
def createMLPClassifier(X, y, x_test, y_test, capas, activaciones, alpha=0.1, epochs=5):
    
    assert len(capas) == len(activaciones) + 1, "Número de activaciones debe coincidir con número de capas"

    n_clases = len(np.unique(y))
    perceptron = PerceptronMulticapa(capas=capas, activaciones=activaciones, alpha=0.01)

    start_time = time.perf_counter()
    perceptron.entrenar(X, np.eye(n_clases)[y], epochs=epochs)
    end_time = time.perf_counter()
    
    # Hacer predicciones sobre el conjunto de prueba
    predicciones = []
    for i in range(len(x_test)):
        prediccion = perceptron.predecir(x_test[i])
        prediccion_clase = np.argmax(prediccion)
        predicciones.append(prediccion_clase)
    
    # Calcular la precisión de las predicciones
    precision = precision_score(y_test, predicciones, average='macro')
    recall = recall_score(y_test , predicciones, average='macro')
    f1 = f1_score(y_test, predicciones, average='macro')
    accuracy = accuracy_score(y_test, predicciones)
    print(f"Precisión: {precision}")
    print(f"Recall: {recall}")
    print(f"F1: {f1}")
    print(f"Exactitud: {accuracy}")
    print(f"Tiempo de entrenamiento: {end_time - start_time}")

# MLP con Tensorflow

In [35]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(shape=[784]))
model.add(tf.keras.layers.Dense(20, activation="sigmoid"))
model.add(tf.keras.layers.Dense(61, activation="sigmoid"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [36]:
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=["accuracy"])

In [37]:
start_time = time.perf_counter()
history = model.fit(
    X_entrenamiento,
    y_entrenamiento,
    epochs=50,
    batch_size=64,
    verbose=2,
    validation_split=0.2
)
end_time = time.perf_counter()

Epoch 1/50
700/700 - 1s - 2ms/step - accuracy: 0.2872 - loss: 2.2474 - val_accuracy: 0.3669 - val_loss: 2.1629
Epoch 2/50
700/700 - 1s - 1ms/step - accuracy: 0.4809 - loss: 2.0677 - val_accuracy: 0.4994 - val_loss: 1.9638
Epoch 3/50
700/700 - 1s - 1ms/step - accuracy: 0.5503 - loss: 1.8375 - val_accuracy: 0.5710 - val_loss: 1.7128
Epoch 4/50
700/700 - 1s - 1ms/step - accuracy: 0.6181 - loss: 1.5837 - val_accuracy: 0.6439 - val_loss: 1.4664
Epoch 5/50
700/700 - 1s - 1ms/step - accuracy: 0.6873 - loss: 1.3543 - val_accuracy: 0.7132 - val_loss: 1.2574
Epoch 6/50
700/700 - 1s - 1ms/step - accuracy: 0.7405 - loss: 1.1646 - val_accuracy: 0.7572 - val_loss: 1.0877
Epoch 7/50
700/700 - 1s - 2ms/step - accuracy: 0.7772 - loss: 1.0113 - val_accuracy: 0.7878 - val_loss: 0.9508
Epoch 8/50
700/700 - 1s - 1ms/step - accuracy: 0.8066 - loss: 0.8875 - val_accuracy: 0.8124 - val_loss: 0.8395
Epoch 9/50
700/700 - 1s - 2ms/step - accuracy: 0.8301 - loss: 0.7862 - val_accuracy: 0.8338 - val_loss: 0.7479
E

In [38]:
model_predictions = model.predict(X_prueba, verbose=0)
model_predictions = model_predictions.argmax(axis=1)

In [39]:
y_prueba_labels = y_prueba.argmax(axis=1)

precision = precision_score(y_prueba_labels, model_predictions, average='macro')
recall = recall_score(y_prueba_labels, model_predictions, average='macro')
f1 = f1_score(y_prueba_labels, model_predictions, average='macro')
accuracy = accuracy_score(y_prueba_labels, model_predictions)

print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Exactitud: {accuracy:.4f}")
print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")

Precisión: 0.9195
Recall: 0.9194
F1-score: 0.9193
Exactitud: 0.9204
Tiempo de entrenamiento: 48.18 segundos
